In [1]:
import torch
import torch.nn as nn
from torch import tensor
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from datasets import load_dataset
from torchvision.transforms import v2

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
trainset = load_dataset('Maysee/tiny-imagenet', split='train')
testset = load_dataset('Maysee/tiny-imagenet', split='valid')

In [3]:
trainset.set_format(type="torch", columns=["image", "label"])
testset.set_format(type="torch", columns=["image", "label"])

In [10]:
img = torch.rand((64,64))
img.repeat(int(3/img.shape[0]))
img.shape

RuntimeError: Number of dimensions of repeat dims can not be smaller than number of dimensions of tensor

In [4]:
# transformation = np.vectorize(lambda x: x.repeat(int(3/x.shape[0])))
for i in range(len(testset)):
    image = trainset[i]['image']
    shape = image.shape
    if shape == torch.Size([64, 64]):
        testset[i]['image'] = image

IndexError: tuple index out of range

In [89]:
# idx_odd_shape = []
for i in range(len(trainset)):
    image = trainset[i]['image']
    shape = image.shape
    if shape == torch.Size([64, 64]):
        image = image.reshape((64, 64,1))
        trainset[i]['image'] = image
        # idx_odd_shape.append(i)

In [82]:
# for index in idx_odd_shape:
#     trainset.remove(trainset[index]['image'])
#     trainset.remove(trainset[index]['label'])

In [90]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,shuffle=True, num_workers=0)

In [91]:
for i, data in enumerate(trainloader, 0):
    image = data['image']
    print(image.shape)

torch.Size([4, 64, 64, 3])
torch.Size([4, 64, 64, 3])
torch.Size([4, 64, 64, 3])
torch.Size([4, 64, 64, 3])
torch.Size([4, 64, 64, 3])
torch.Size([4, 64, 64, 3])
torch.Size([4, 64, 64, 3])
torch.Size([4, 64, 64, 3])


RuntimeError: stack expects each tensor to be equal size, but got [64, 64, 3] at entry 0 and [64, 64] at entry 1

## Choose a Device

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  

In [53]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv_layer_one = nn.Conv2d(in_channels=3,out_channels=1024,kernel_size=3 , stride=1, padding=1)
        self.conv_layer_two = nn.Conv2d(in_channels=1024,out_channels=720,kernel_size=3, stride=1, padding=1)
        self.conv_layer_three = nn.Conv2d(in_channels=720,out_channels=440,kernel_size=3, stride=1, padding=1)
        self.conv_layer_four = nn.Conv2d(in_channels=440,out_channels=200,kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(3200,2480)
        self.fc2 = nn.Linear(2480,1680)
        self.fc3 = nn.Linear(1680,980)
        self.fc4 = nn.Linear(980,640)
        
    def forward(self, x):
        x = self.conv_layer_one(x)
        x = F.relu(x)
        x = F.max_pool2d(x,(2,2))
        x = self.conv_layer_two(x)
        x = F.relu(x)
        x = F.avg_pool2d(x,(2,2))
        x = self.conv_layer_three(x)
        x = F.relu(x)
        x = F.max_pool2d(x,(2,2))
        x = self.conv_layer_four(x)
        x = F.relu(x)
        x = F.max_pool2d(x,(2,2))
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.fc4(x)
        return x
    

net = Net()     # Create the network instance.
net.to(device)  # Move the network parameters to the specified device.

(100000, 2)

## Choose Loss Function

In [ ]:
loss_func = nn.CrossEntropyLoss()  
opt = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) 

In [ ]:
avg_losses = []   # Avg. losses.
epochs = 25       # Total epochs.
print_freq = 10000  # Print frequency.

for epoch in range(epochs):  # Loop over the dataset multiple times.
    running_loss = 0.0       # Initialize running loss.
    for i, data in enumerate(trainloader, 0):
        # Get the inputs.
        inputs, labels = data
        
        # Move the inputs to the specified device.
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients.
        opt.zero_grad()

        # Forward step.
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        
        # Backward step.
        loss.backward()
        
        # Optimization step (update the parameters).
        opt.step()

        # Print statistics.
        running_loss += loss.item()
        if i % print_freq == print_freq - 1: # Print every several mini-batches.
            avg_loss = running_loss / print_freq
            print('[epoch: {}, i: {:5d}] avg mini-batch loss: {:.3f}'.format(
                epoch, i, avg_loss))
            avg_losses.append(avg_loss)
            running_loss = 0.0

print('Finished Training.')

## Plot Training Loss

In [ ]:
plt.plot(avg_losses)
plt.xlabel('mini-batch index / {}'.format(print_freq))
plt.ylabel('avg. mini-batch loss')
plt.show()

## Evaluate on Test Set

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        print("predicted")
        print(predicted)
        print("labels")
        print(labels)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))